# PKS1510-089 analysis notebook
Reference paper: https://www.aanda.org/articles/aa/pdf/2011/05/aa16128-10.pdf

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
# Interactive plots
#%matplotlib widget

## Creating a configuration file

In [ ]:
confFilePath = "$HOME/agilepy_conf.yaml"

In [ ]:
AGAnalysis.getConfiguration(
    confFilePath = confFilePath,
    evtfile="/AGILE_PROC3/FM3.119_ASDCe/INDEX/EVT.index",
    logfile="/AGILE_PROC3/DATA_ASDCe/INDEX/LOG.log.index",
    userName = "username",
    sourceName = "PKS1510-089",
    tmin = 54891,
    tmax = 54921,
    timetype = "MJD",
    glon = 351.29,
    glat = 40.13,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 0
)

## Obtaining the AGAnalysis object

In [ ]:
ag = AGAnalysis(confFilePath)

In [ ]:
ag.setOptions(binsize=0.5)

In [ ]:
#print all options of the configuration file
ag.printOptions()

## Sources hypothesis

In [ ]:
#sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 5), show=True)

#aggiungeto una sorgente in questo modo non funziona più il selectSources
newSourceDict = {
    "glon" : 351.29,
    "glat": 40.13,
    "spectrumType" : "PowerLaw",
    "flux": 2.0e-07,
    "index": 2.1
}
newSource = ag.addSource("PKS1510-089", newSourceDict)

## Selecting sources

In [ ]:
sources = ag.selectSources("flux > 0", show = True)

## Free a source's parameter
Freeable params = [flux, index, index1, index2, cutoffEnergy, pivotEnergy, curvature, pos]

In [ ]:
affectedSources = ag.freeSources('name == "PKS1510-089"', "flux", True, show=True)

In this example, only one source is affected.

## Analysis

In [ ]:
#Generate maps
maplistfile = ag.generateMaps()

In [ ]:
#Display counts map
ag.displayCtsSkyMaps(maplistFile=maplistfile, smooth=3, catalogRegions="2AGL", catalogRegionsColor="green")

In [ ]:
#Fix the diffuse emission coefficient to a default value. 
ag.setOptions(galcoeff=[0.7])

In [ ]:
ag.printOptions("model")

In [ ]:
#Perform a maximum likelihood estimator
ag.mle()

In [ ]:
#Display the results
ag.selectSources("sqrtTS > 0", show=True)

## Spectral index evaluation

In [ ]:
#energybin=2 to calcolate spectral index
ag.setOptionEnergybin(2)
ag.setOptions(galcoeff=[0.7, 0.7, 0.7, 0.7])
ag.setOptions(isocoeff=[-1, -1, -1, -1])

In [ ]:
maplistfile = ag.generateMaps()

In [ ]:
ag.displayCtsSkyMaps(maplistFile=maplistfile, smooth=3, normType="power", catalogRegions="2AGL", catalogRegionsColor="green")

In [ ]:
affectedSources = ag.freeSources('name == "PKS1510-089"', "index", True, show=True)

In [ ]:
ag.mle()

In [ ]:
selectedSources = ag.selectSources('flux > 0', show=True)

In [ ]:
ag.setOptionEnergybin(0)
ag.setOptions(galcoeff=[0.7])
affectedSources = ag.freeSources('name == "PKS1510-089"', "index", False, show=True)

### Light curve with default tmin and tmax

In [ ]:
lightCurveData1 = ag.lightCurveMLE("PKS1510-089", binsize=86400)

In [ ]:
cat $lightCurveData1

In [ ]:
print(lightCurveData1)

In [ ]:
ag.displayLightCurve("mle")

In [ ]:
affectedSources = ag.freeSources('name == "PKS1510-089"', "pos", True, show=True)
lightCurveData1 = ag.lightCurveMLE("PKS1510-089", binsize=86400)

In [ ]:
ag.displayLightCurve("mle")

In [ ]:
lightCurveData4 = ag.lightCurveMLE("PKS1510-089", binsize=86400*4)

In [ ]:
ag.displayLightCurve("mle")

In [ ]:
ag.selectSources('name == "PKS1510-089"', show=True)

time_start_mjd time_end_mjd sqrt(ts) flux flux_err flux_ul gal gal_error iso iso_error l_peak b_peak dist_peak l b r ell_dist a b phi exposure ExpRatio counts counts_err Index Index_Err Par2 Par2_Err Par3 Par3_Err Erglog Erglog_Err Erglog_UL time_start_utc time_end_utc time_start_tt time_end_tt Fix index ULConfidenceLevel SrcLocConfLevel start_l start_b start_flux typefun par2 par3 galmode2 galmode2fit isomode2 isomode2fit edpcor fluxcor integratortype expratioEval expratio_minthr expratio_maxthr expratio_size Emin emax fovmin fovmax albedo binsize expstep phasecode

In [ ]:
ag.displayGenericColumns(lightCurveData4, ["ExpRatio"], um=[""])

## Cleaning up

In [ ]:
#ag.deleteAnalysisDir()

# 

##### 